**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import skvideo
import cv2
import json

from keras.models import Sequential,model_from_json, Model
from keras.layers import Input, Flatten
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act is where the agent takes an action, i.e. give a state of the environmnent, it decides of an action to perfom on the environment, leading to the next state, and receiving a reward for this action.

There is a tradeoff in reinforcement learning between exploration and exploitation. Exploration corresponds to the random actions the agent takes in order to explore all the possible states. Exploitation corresponds to the optimal action to take given what the agent learned. ```Epsilon``` is this hyperparameter controlling the proportion of random action to perform, thus how much we want the agent to explore. It is essential, because the agent would otherwise heavily depend on the initial state, and would fall onto sub-optimal solutions.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

`position` and `board` are two 2D arrays of size $(N+2, N+2)$. 

`position` is filled with $0$ everywhere on the actual island, padded with $-1$ on the borders (2 lines). It corresponds to the reachable positions by the agent, and also stores, with a value equal to $1$, the position of the rat before an action is taken.

`board` stores the remaining occupied cells in the island, with randomly sampled values of $0.5$ (cheese) and $-1$ (poison). When the rat occupies a cell, its value is set to $0$.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(self.n_action)
        return action
        

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # Reset env at the beginning of each episode
        state = env.reset()
        
        game_over = False
        
        win = 0
        lose = 0
        
        while not game_over:
                
            # Sample action
            action = agent.act(state)
        
            # Apply action to env - receive reward, state, info
            prev_state = state
            state, reward, game_over = env.act(action)
        
            # Update the counters
            if reward > 0:
                win += reward
            elif reward < 0:
                lose -= reward
        
        if e % 10 == 0:
            env.draw(e)
        
        score += win-lose
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [15]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()
skvideo.setFFmpegPath("/usr/local/bin/")
test(agent,env,2,prefix='random')
HTML(display_videos('random1.mp4'))

Win/lose count 9.5/12.0. Average score (-5.0)
Win/lose count 4.5/8.0. Average score (-6.0)
Final score: -6.0


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Isolating the first term in the value function, we obtain : 

\begin{equation} Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \ = \ r(s,a) + \gamma E_{p^{\pi}}[\sum_{t \geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \ = r(s,a) + \gamma E_{(s',a')\sim p(.|s,a)}[\sum_{t \geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'] \ = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \end{equation}

Then, it comes directly, using a deterministic policy :

\begin{equation} Q^{*}(s,a) = \max_{\pi} Q^\pi(s,a) = Q^{\pi^{*}}(s,a) \ = r(s,a) + \gamma E_{(s',a')\sim p^{\pi^{*}}(.|s,a)} [ Q^{\pi^{*}}(s^{'},a^{'})] \ = r(s,a) + \gamma E_{s' \sim \pi^{*}(.|s,a)} [ \max_{a^{'}} Q^{*}(s^{'},a^{'})] \end{equation}

The final loss measure the difference between the two previous equations, thus measing the gap between the value obtain by an hypothetical optimal policy and the actual policy. 

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) == self.max_memory + 1:
            self.memory.pop(0)

    def random_access(self):
        output = self.memory[np.random.randint(len(self.memory))]
        return output

***
The pipeline we will use for training is given below:

In [12]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [20]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s]))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            
            input_states[i] = s_

            target_q[i] = self.model.predict(np.array([s_]))[0]
            
            if game_over_:
                target_q[i, a_] = r_
                
            else:
                target_q[i, a_] = r_ + self.discount * self.model.predict(n_s_.reshape(1, 5, 5, self.n_state)).max()


        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        inp = Input(shape=(5, 5, self.n_state))
        x = Flatten()(inp)
        x = Dense(64, activation="relu")(x)
        x = Dense(4, activation="linear")(x)
        model = Model(inputs=inp, outputs=x)
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [19]:
epochs_train = 61
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

Epoch 000/061 | Loss 0.0034 | Win/lose count 1.5/5.0 (-3.5)
Epoch 001/061 | Loss 0.0034 | Win/lose count 4.0/2.0 (2.0)
Epoch 002/061 | Loss 0.0028 | Win/lose count 6.0/6.0 (0.0)
Epoch 003/061 | Loss 0.0034 | Win/lose count 5.5/3.0 (2.5)
Epoch 004/061 | Loss 0.0020 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/061 | Loss 0.0029 | Win/lose count 5.5/8.0 (-2.5)
Epoch 006/061 | Loss 0.0075 | Win/lose count 7.5/2.0 (5.5)
Epoch 007/061 | Loss 0.0034 | Win/lose count 7.0/2.0 (5.0)
Epoch 008/061 | Loss 0.0023 | Win/lose count 4.0/4.0 (0.0)
Epoch 009/061 | Loss 0.0022 | Win/lose count 5.0/3.0 (2.0)
Epoch 010/061 | Loss 0.0038 | Win/lose count 9.5/0 (9.5)
Epoch 011/061 | Loss 0.0044 | Win/lose count 3.0/2.0 (1.0)
Epoch 012/061 | Loss 0.0029 | Win/lose count 9.5/2.0 (7.5)
Epoch 013/061 | Loss 0.0020 | Win/lose count 5.0/3.0 (2.0)
Epoch 014/061 | Loss 0.0023 | Win/lose count 7.0/3.0 (4.0)
Epoch 015/061 | Loss 0.0062 | Win/lose count 8.5/7.0 (1.5)
Epoch 016/061 | Loss 0.0023 | Win/lose count 8.5/3.0 (5.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [21]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        inp = Input(shape=(5, 5, self.n_state))
        x = Conv2D(16, (3, 3), activation='relu')(inp)
        x = Conv2D(16, (3, 3), activation='relu')(x)
        x = Flatten()(x)
        x = Dense(4, activation="linear")(x) 

        model = Model(inputs=inp, outputs=x)
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [23]:
epochs_train = 91
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train90.mp4'))

Epoch 000/091 | Loss 0.0010 | Win/lose count 2.0/5.0 (-3.0)
Epoch 001/091 | Loss 0.0020 | Win/lose count 4.5/4.0 (0.5)
Epoch 002/091 | Loss 0.0046 | Win/lose count 6.5/5.0 (1.5)
Epoch 003/091 | Loss 0.0085 | Win/lose count 10.5/7.0 (3.5)
Epoch 004/091 | Loss 0.0408 | Win/lose count 9.0/7.0 (2.0)
Epoch 005/091 | Loss 0.0073 | Win/lose count 9.0/5.0 (4.0)
Epoch 006/091 | Loss 0.0044 | Win/lose count 4.0/4.0 (0.0)
Epoch 007/091 | Loss 0.0030 | Win/lose count 6.0/4.0 (2.0)
Epoch 008/091 | Loss 0.0035 | Win/lose count 7.5/6.0 (1.5)
Epoch 009/091 | Loss 0.0044 | Win/lose count 7.5/5.0 (2.5)
Epoch 010/091 | Loss 0.0018 | Win/lose count 4.0/2.0 (2.0)
Epoch 011/091 | Loss 0.0027 | Win/lose count 9.0/2.0 (7.0)
Epoch 012/091 | Loss 0.0022 | Win/lose count 2.0/6.0 (-4.0)
Epoch 013/091 | Loss 0.0031 | Win/lose count 5.0/4.0 (1.0)
Epoch 014/091 | Loss 0.0019 | Win/lose count 11.5/1.0 (10.5)
Epoch 015/091 | Loss 0.0034 | Win/lose count 6.0/1.0 (5.0)
Epoch 016/091 | Loss 0.0043 | Win/lose count 8.5/2.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [24]:
epochs_test = 10
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 10.0/2.0. Average score (16.0)
Win/lose count 8.5/3.0. Average score (13.5)
Win/lose count 9.0/4.0. Average score (12.333333333333334)
Win/lose count 5.5/3.0. Average score (10.5)
Win/lose count 3.0/2.0. Average score (8.8)
Win/lose count 11.0/2.0. Average score (10.333333333333334)
Win/lose count 6.5/0. Average score (10.714285714285714)
Win/lose count 6.5/1.0. Average score (10.75)
Win/lose count 12.0/4.0. Average score (11.333333333333334)
Win/lose count 8.5/2.0. Average score (11.5)
Final score: 11.5
Test of the FC
Win/lose count 1.0/9.0. Average score (-16.0)
Win/lose count 3.0/7.0. Average score (-12.0)
Win/lose count 4.0/6.0. Average score (-9.333333333333334)
Win/lose count 4.0/5.0. Average score (-7.5)
Win/lose count 2.0/9.0. Average score (-8.8)
Win/lose count 5.0/7.0. Average score (-8.0)
Win/lose count 4.0/5.0. Average score (-7.142857142857143)
Win/lose count 2.5/4.0. Average score (-6.625)
Win/lose count 2.0/6.0. Average score (-6.7777777777

**Comment** Both agents are often trapped into loops, or goes back and forth. Additionnaly, the agent do not explore the entire island. The CNN provides best results.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [90]:
def train_explore(agent, env, epoch, epsilon_decay, decay_step, min_epsilon, prefix=''):

    score = 0
    loss = 0
    for e in range(epoch):
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        eps_decay = False
        while not game_over:
            #print(f'decay:{decay_step}, e:{e + 1}, bool: {(e + 1) % decay_step == 0}')
            if (e + 1) % decay_step == 0 and not eps_decay:
                eps_ = agent.epsilon/epsilon_decay
                if eps_ >= min_epsilon:
                    agent.set_epsilon(eps_)
                    eps_decay = True
                    print(agent.epsilon)
                    

            action = agent.act(state)

            prev_state = state
            state, reward, game_over = env.act(action)

            if reward > 0 :
                win += reward
            else:
                lose -= reward
            
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        if e % 10 == 0:
            env.draw(prefix+str(e))

        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
     
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, penalty=0.1):
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)
        
        # Penalize if agent goies to a visited cell
        self.penalty = penalty
        self.penalty_grid = np.zeros_like(self.board)

        
    def reset(self):
        # Reseting base class
        super(EnvironmentExploring, self).reset()
        
        # Resetting penalty 
        self.penalty_grid = np.zeros_like(self.board)
        
        state = np.concatenate((self.penalty_grid.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
        
    
    def act(self, action):
        
        state, reward, game_over = super(EnvironmentExploring, self).act(action)

        # Penalized if already visited
        if train:
            reward -= self.penalty_grid[self.x, self.y]
            self.penalty_grid[self.x, self.y] = self.penalty

        # 3 "feature" states instead of 2
        state = np.concatenate((self.penalty_grid.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over
    
    # Update video frame 
    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[self.penalty_grid > 0, :] /= 2 # Darken visited cells
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

In [92]:
# Training
epochs_train, epsilon, epsilon_decay, decay_step, min_epsilon = 151, .4, 2, 20, 0.01
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3, penalty=0.05)
agent = DQN_CNN(size, lr=.1, epsilon = epsilon, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, epsilon_decay, decay_step, min_epsilon, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore150.mp4'))

Epoch 000/151 | Loss 0.0748 | Win/lose count 12.0/27.700000000000063 (-15.700000000000063)
Epoch 001/151 | Loss 0.0216 | Win/lose count 15.5/26.550000000000068 (-11.050000000000068)
Epoch 002/151 | Loss 0.0329 | Win/lose count 11.0/20.05000000000005 (-9.05000000000005)
Epoch 003/151 | Loss 0.0348 | Win/lose count 14.0/27.350000000000065 (-13.350000000000065)
Epoch 004/151 | Loss 0.0143 | Win/lose count 12.0/17.70000000000006 (-5.70000000000006)
Epoch 005/151 | Loss 0.0440 | Win/lose count 12.5/14.500000000000036 (-2.0000000000000355)
Epoch 006/151 | Loss 0.0371 | Win/lose count 13.0/23.45000000000008 (-10.450000000000081)
Epoch 007/151 | Loss 0.0352 | Win/lose count 12.5/20.55000000000005 (-8.05000000000005)
Epoch 008/151 | Loss 0.0690 | Win/lose count 5.5/19.100000000000097 (-13.600000000000097)
Epoch 009/151 | Loss 0.0170 | Win/lose count 12.0/21.000000000000064 (-9.000000000000064)
Epoch 010/151 | Loss 0.0186 | Win/lose count 11.5/20.95000000000003 (-9.450000000000031)
Epoch 011/151

Epoch 091/151 | Loss 0.0251 | Win/lose count 12.0/20.300000000000058 (-8.300000000000058)
Epoch 092/151 | Loss 0.0329 | Win/lose count 12.0/16.400000000000066 (-4.400000000000066)
Epoch 093/151 | Loss 0.0133 | Win/lose count 9.0/22.05000000000007 (-13.050000000000072)
Epoch 094/151 | Loss 0.0215 | Win/lose count 6.5/18.650000000000027 (-12.150000000000027)
Epoch 095/151 | Loss 0.0257 | Win/lose count 15.0/17.10000000000006 (-2.1000000000000583)
Epoch 096/151 | Loss 0.0172 | Win/lose count 10.0/24.20000000000008 (-14.200000000000081)
Epoch 097/151 | Loss 0.0255 | Win/lose count 12.0/16.350000000000055 (-4.350000000000055)
Epoch 098/151 | Loss 0.0371 | Win/lose count 11.0/23.550000000000086 (-12.550000000000086)
0.0125
Epoch 099/151 | Loss 0.0198 | Win/lose count 10.0/21.000000000000068 (-11.000000000000068)
Epoch 100/151 | Loss 0.0291 | Win/lose count 5.5/16.350000000000033 (-10.850000000000033)
Epoch 101/151 | Loss 0.0190 | Win/lose count 12.5/14.350000000000062 (-1.8500000000000618)
E

In [62]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 1.0/12.300000000000043. Average score (-22.600000000000087)
Win/lose count 0.5/14.250000000000039. Average score (-25.050000000000082)
Win/lose count 3.0/13.300000000000047. Average score (-23.56666666666675)
Win/lose count 1.5/10.500000000000014. Average score (-22.17500000000007)
Win/lose count 2.5/12.40000000000003. Average score (-21.70000000000007)
Win/lose count 1.5/9.5. Average score (-20.75000000000006)
Win/lose count 1.0/15.100000000000076. Average score (-21.814285714285784)
Win/lose count 1.0/12.50000000000005. Average score (-21.962500000000077)
Win/lose count 2.0/10.400000000000016. Average score (-21.388888888888957)
Win/lose count 2.0/13.20000000000006. Average score (-21.490000000000073)
Final score: -21.490000000000073


***